In [1]:
# Install ChromaDB (run this cell once)
!pip install chromadb
import pandas as pd
import chromadb
from chromadb.config import Settings
import ast
import json
from IPython.display import display, HTML

print("✅ Libraries imported successfully!")

✅ Libraries imported successfully!


In [14]:
# Initialize ChromaDB client
chroma_client = chromadb.PersistentClient(path="./chroma_db")

# Create or get collection
collection = chroma_client.get_or_create_collection(
    name="video_embeddings",
    metadata={"description": "Video transcripts and titles with embeddings"}
)

print("✅ ChromaDB client initialized!")
print(f"Collection count: {collection.count()}")

✅ ChromaDB client initialized!
Collection count: 0


In [15]:
# Load your CSV file
df = pd.read_csv("dataset_with_embeddings.csv")  # Replace with your actual file path

# Display first few rows to verify
print("📊 First 5 rows of your data:")
display(df.head())

print(f"\n📈 Dataset shape: {df.shape}")
print(f"Columns: {list(df.columns)}")

📊 First 5 rows of your data:


,id,title,description,publishedAt,tags,categoryId,defaultLanguage,defaultAudioLanguage,thumbnail_default,thumbnail_high,...,channel_title,channel_description,channel_country,channel_thumbnail,channel_subscriberCount,channel_videoCount,is_transcript_available,transcript,combined_text,embeddings
0,t8txtQkhMcY,Scientists Can’t Explain What’s Happening on T...,High in Tibet stands a mountain no one has eve...,2025-10-28T06:45:00Z,"ancient mysteries, ancient secrets, brightside...",24.0,en,en-US,https://i.ytimg.com/vi/t8txtQkhMcY/default.jpg,https://i.ytimg.com/vi/t8txtQkhMcY/hqdefault.jpg,...,BRIGHT SIDE,"Whether you’re into recent discoveries, space ...",NaN,https://yt3.ggpht.com/BimIuq2UYH1h-GQDEiV89I1d...,44700000.0,10619.0,True,Scientists Canâ€™t Explain Whatâ€™s Happening ...,Scientists Can’t Explain What’s Happening on T...,"[-0.12879705429077148, 0.09205026179552078, 0...."
1,mmIidOoHCfs,$138 Million Pirate Treasure Found on the Ocea...,A team of explorers just uncovered a real pira...,2025-10-22T20:00:41Z,"ancient shipwreck, brightside, brightsidegloba...",24.0,en-US,en-US,https://i.ytimg.com/vi/mmIidOoHCfs/default.jpg,https://i.ytimg.com/vi/mmIidOoHCfs/hqdefault.jpg,...,BRIGHT SIDE,"Whether you’re into recent discoveries, space ...",NaN,https://yt3.ggpht.com/BimIuq2UYH1h-GQDEiV89I1d...,44700000.0,10619.0,True,$138 Million Pirate Treasure Found on the Ocea...,$138 Million Pirate Treasure Found on the Ocea...,"[-0.09365776181221008, 0.11550658941268921, -0..."
2,QBHjeGGtIKE,"If You See a Rubber Band on Your Door, Act Fast!",Strange things like a rubber band on your door...,2025-10-22T18:16:20Z,"brightside, brightsideglobal, burglary prevent...",24.0,en,en-US,https://i.ytimg.com/vi/QBHjeGGtIKE/default.jpg,https://i.ytimg.com/vi/QBHjeGGtIKE/hqdefault.jpg,...,BRIGHT SIDE,"Whether you’re into recent discoveries, space ...",NaN,https://yt3.ggpht.com/BimIuq2UYH1h-GQDEiV89I1d...,44700000.0,10619.0,True,If You See a Rubber Bads,"If You See a Rubber Band on Your Door, Act Fas...","[0.048036929219961166, -0.08153478801250458, 0..."
3,DvxiRgk2RZI,Cleopatra’s Tomb Found? Egypt’s Greatest Disco...,An archaeologist believes Cleopatra's lost tom...,2025-10-22T18:00:13Z,"12000 year old tomb, 5 most shocking discoveri...",24.0,en,en,https://i.ytimg.com/vi/DvxiRgk2RZI/default.jpg,https://i.ytimg.com/vi/DvxiRgk2RZI/hqdefault.jpg,...,BRIGHT SIDE,"Whether you’re into recent discoveries, space ...",NaN,https://yt3.ggpht.com/BimIuq2UYH1h-GQDEiV89I1d...,44700000.0,10619.0,True,Cleopatraâ€™s Tomb Found? Egyptâ€™s,Cleopatra’s Tomb Found? Egypt’s Greatest Disco...,"[-0.05367204174399376, 0.11010417342185974, -0..."
4,CvTsL-WcrgQ,Scientists Just Solved the Mystery of Killer W...,"For centuries, sailors told terrifying stories...",2025-10-22T00:00:25Z,"brightside, brightsideglobal, curiosities, dan...",24.0,en,en,https://i.ytimg.com/vi/CvTsL-WcrgQ/default.jpg,https://i.ytimg.com/vi/CvTsL-WcrgQ/hqdefault.jpg,...,BRIGHT SIDE,"Whether you’re into recent discoveries, space ...",NaN,https://yt3.ggpht.com/BimIuq2UYH1h-GQDEiV89I1d...,44700000.0,10619.0,True,Scientists Just Solved the Mystery of Killer W...,Scientists Just Solved the Mystery of Killer W...,"[-0.07009203732013702, 0.056527212262153625, 0..."



📈 Dataset shape: (622, 26)
Columns: ['id', 'title', 'description', 'publishedAt', 'tags', 'categoryId', 'defaultLanguage', 'defaultAudioLanguage', 'thumbnail_default', 'thumbnail_high', 'duration', 'viewCount', 'likeCount', 'commentCount', 'privacyStatus', 'channel_id', 'channel_title', 'channel_description', 'channel_country', 'channel_thumbnail', 'channel_subscriberCount', 'channel_videoCount', 'is_transcript_available', 'transcript', 'combined_text', 'embeddings']


In [18]:
def add_videos_to_chromadb(df):
    """
    Read DataFrame and add all data to ChromaDB
    """
    # Prepare data for ChromaDB
    documents = []
    metadatas = []
    ids = []
    embeddings = []
    
    for index, row in df.iterrows():
        # Combine title and transcript
        combined_text = f"{row['title']} {row['transcript']}"
        
        # Convert embedding string back to list (if stored as string)
        if isinstance(row['embeddings'], str):
            try:
                embedding = ast.literal_eval(row['embeddings'])
            except:
                # If literal_eval fails, try JSON load
                embedding = json.loads(row['embeddings'])
        else:
            embedding = row['embeddings']
        
        # Prepare metadata
        metadata = {
            'title': row['title'],
            'channel_title': row['channel_title'],
            'view_count': int(row['view_count']),
            'duration': row['duration'],
            'original_video_id': row['video_id']
        }
        
        documents.append(combined_text)
        embeddings.append(embedding)
        metadatas.append(metadata)
        ids.append(f"video_{index}")
    
    # Add to ChromaDB
    collection.add(
        embeddings=embeddings,
        documents=documents,
        metadatas=metadatas,
        ids=ids
    )
    
    print(f"✅ Successfully added {len(ids)} videos to ChromaDB")
    return len(ids)

In [19]:
# Debug: Check your DataFrame structure
print("🔍 Debugging - Checking DataFrame:")
print(f"DataFrame shape: {df.shape}")
print(f"Columns: {list(df.columns)}")
print(f"First few rows:")
display(df.head(2))

# Check if embeddings column exists and sample content
if 'embeddings' in df.columns:
    print(f"\n📊 Embeddings column sample:")
    print(f"Type of first embedding: {type(df['embeddings'].iloc[0])}")
    print(f"First embedding preview: {str(df['embeddings'].iloc[0])[:100]}...")
else:
    print("❌ 'embeddings' column not found!")
    print(f"Available columns: {list(df.columns)}")

🔍 Debugging - Checking DataFrame:
DataFrame shape: (622, 26)
Columns: ['id', 'title', 'description', 'publishedAt', 'tags', 'categoryId', 'defaultLanguage', 'defaultAudioLanguage', 'thumbnail_default', 'thumbnail_high', 'duration', 'viewCount', 'likeCount', 'commentCount', 'privacyStatus', 'channel_id', 'channel_title', 'channel_description', 'channel_country', 'channel_thumbnail', 'channel_subscriberCount', 'channel_videoCount', 'is_transcript_available', 'transcript', 'combined_text', 'embeddings']
First few rows:


,id,title,description,publishedAt,tags,categoryId,defaultLanguage,defaultAudioLanguage,thumbnail_default,thumbnail_high,...,channel_title,channel_description,channel_country,channel_thumbnail,channel_subscriberCount,channel_videoCount,is_transcript_available,transcript,combined_text,embeddings
0,t8txtQkhMcY,Scientists Can’t Explain What’s Happening on T...,High in Tibet stands a mountain no one has eve...,2025-10-28T06:45:00Z,"ancient mysteries, ancient secrets, brightside...",24.0,en,en-US,https://i.ytimg.com/vi/t8txtQkhMcY/default.jpg,https://i.ytimg.com/vi/t8txtQkhMcY/hqdefault.jpg,...,BRIGHT SIDE,"Whether you’re into recent discoveries, space ...",NaN,https://yt3.ggpht.com/BimIuq2UYH1h-GQDEiV89I1d...,44700000.0,10619.0,True,Scientists Canâ€™t Explain Whatâ€™s Happening ...,Scientists Can’t Explain What’s Happening on T...,"[-0.12879705429077148, 0.09205026179552078, 0...."
1,mmIidOoHCfs,$138 Million Pirate Treasure Found on the Ocea...,A team of explorers just uncovered a real pira...,2025-10-22T20:00:41Z,"ancient shipwreck, brightside, brightsidegloba...",24.0,en-US,en-US,https://i.ytimg.com/vi/mmIidOoHCfs/default.jpg,https://i.ytimg.com/vi/mmIidOoHCfs/hqdefault.jpg,...,BRIGHT SIDE,"Whether you’re into recent discoveries, space ...",NaN,https://yt3.ggpht.com/BimIuq2UYH1h-GQDEiV89I1d...,44700000.0,10619.0,True,$138 Million Pirate Treasure Found on the Ocea...,$138 Million Pirate Treasure Found on the Ocea...,"[-0.09365776181221008, 0.11550658941268921, -0..."



📊 Embeddings column sample:
Type of first embedding: <class 'str'>
First embedding preview: [-0.12879705429077148, 0.09205026179552078, 0.10372598469257355, 0.05230065435171127, -0.02730874158...


In [21]:
# Debug: Check ALL your columns to find the embedding column
print("🔍 Checking ALL columns in your DataFrame:")
for i, col in enumerate(df.columns):
    print(f"  {i+1:2d}. '{col}'")

print(f"\n📊 DataFrame shape: {df.shape}")
print(f"Total columns: {len(df.columns)}")

# Look for any column that might contain embeddings
embedding_like_columns = [col for col in df.columns if 'embed' in col.lower() or 'vector' in col.lower()]
print(f"\n🎯 Possible embedding columns: {embedding_like_columns}")

# If no obvious embedding columns, let's check the data types and sample content
print(f"\n📋 Sample of each column (first 50 chars):")
for col in df.columns[:10]:  # Check first 10 columns
    sample = str(df[col].iloc[0])[:50] if pd.notna(df[col].iloc[0]) else "NaN"
    print(f"  '{col}': {sample}...")

🔍 Checking ALL columns in your DataFrame:
   1. 'id'
   2. 'title'
   3. 'description'
   4. 'publishedAt'
   5. 'tags'
   6. 'categoryId'
   7. 'defaultLanguage'
   8. 'defaultAudioLanguage'
   9. 'thumbnail_default'
  10. 'thumbnail_high'
  11. 'duration'
  12. 'viewCount'
  13. 'likeCount'
  14. 'commentCount'
  15. 'privacyStatus'
  16. 'channel_id'
  17. 'channel_title'
  18. 'channel_description'
  19. 'channel_country'
  20. 'channel_thumbnail'
  21. 'channel_subscriberCount'
  22. 'channel_videoCount'
  23. 'is_transcript_available'
  24. 'transcript'
  25. 'combined_text'
  26. 'embeddings'

📊 DataFrame shape: (622, 26)
Total columns: 26

🎯 Possible embedding columns: ['embeddings']

📋 Sample of each column (first 50 chars):
  'id': t8txtQkhMcY...
  'title': Scientists Can’t Explain What’s Happening on This ...
  'description': High in Tibet stands a mountain no one has ever co...
  'publishedAt': 2025-10-28T06:45:00Z...
  'tags': ancient mysteries, ancient secrets, brightside

In [22]:
# Quick test with first row - USING CORRECT COLUMN NAME
print("🧪 Testing with first row...")
row = df.iloc[0]

print(f"Title: {row['title']}")
print(f"Embedding column type: {type(row['embeddings'])}")  # FIXED: 'embeddings' not 'embedding'
print(f"Embedding preview: {str(row['embeddings'])[:100]}...")

# Test embedding conversion
if isinstance(row['embeddings'], str):
    try:
        test_embedding = ast.literal_eval(row['embeddings'])
        print(f"✅ Embedding conversion successful! Length: {len(test_embedding)}")
        print(f"✅ First few values: {test_embedding[:5]}...")
    except Exception as e:
        print(f"❌ Embedding conversion failed: {e}")
else:
    print(f"✅ Embedding is already a list, length: {len(row['embeddings'])}")
    print(f"✅ First few values: {row['embeddings'][:5]}...")

🧪 Testing with first row...
Title: Scientists Can’t Explain What’s Happening on This Mountain in Tibet
Embedding column type: <class 'str'>
Embedding preview: [-0.12879705429077148, 0.09205026179552078, 0.10372598469257355, 0.05230065435171127, -0.02730874158...
✅ Embedding conversion successful! Length: 384
✅ First few values: [-0.12879705429077148, 0.09205026179552078, 0.10372598469257355, 0.05230065435171127, -0.027308741584420204]...


In [23]:
def add_videos_to_chromadb(df):
    """
    Read DataFrame and add all data to ChromaDB
    """
    print("🔄 Starting to add videos to ChromaDB...")
    
    # Prepare data for ChromaDB
    documents = []
    metadatas = []
    ids = []
    embeddings_list = []  # Renamed to avoid confusion with column name
    
    successful_rows = 0
    problematic_rows = 0
    
    for index, row in df.iterrows():
        try:
            # Use combined_text column (you already have it!)
            combined_text = row['combined_text']
            
            # CORRECTED: Use 'embeddings' (with 's') - your actual column name
            embedding_value = row['embeddings']
            
            # Convert embedding string back to list (if stored as string)
            if isinstance(embedding_value, str):
                try:
                    embedding = ast.literal_eval(embedding_value)
                except:
                    # If literal_eval fails, try JSON load
                    embedding = json.loads(embedding_value)
            else:
                embedding = embedding_value
            
            # Validate embedding
            if not isinstance(embedding, list):
                print(f"❌ Row {index}: Embedding is not a list, type: {type(embedding)}")
                problematic_rows += 1
                continue
            
            # Prepare metadata - using your actual column names
            metadata = {
                'title': row['title'],
                'channel_title': row['channel_title'],
                'view_count': int(row['viewCount']),  # Note: viewCount not view_count
                'duration': row['duration'],
                'video_id': row['id'],  # Note: id not video_id
                'description': row['description'][:200] if pd.notna(row['description']) else "",  # Truncate long descriptions
                'published_at': row['publishedAt'],
                'like_count': int(row['likeCount']) if pd.notna(row['likeCount']) else 0
            }
            
            documents.append(combined_text)
            embeddings_list.append(embedding)
            metadatas.append(metadata)
            ids.append(f"video_{index}")
            successful_rows += 1
            
            # Print progress for first few rows
            if successful_rows <= 3:
                print(f"✅ Row {index}: Successfully processed - '{row['title'][:30]}...'")
                print(f"   Embedding length: {len(embedding)}")
            
        except Exception as e:
            print(f"❌ Error processing row {index}: {e}")
            problematic_rows += 1
            continue
    
    print(f"\n📊 Summary:")
    print(f"✅ Processed {successful_rows} rows successfully")
    print(f"❌ Had {problematic_rows} problematic rows")
    
    if successful_rows > 0:
        # Add to ChromaDB
        print("💾 Saving to ChromaDB...")
        collection.add(
            embeddings=embeddings_list,
            documents=documents,
            metadatas=metadatas,
            ids=ids
        )
        print(f"🎉 Successfully added {len(ids)} videos to ChromaDB")
        print(f"📊 ChromaDB collection now has {collection.count()} total items")
    else:
        print("⚠️ No data was added to ChromaDB")
    
    return len(ids)

# EXECUTE THE FUNCTION
print("🚀 Executing the function with CORRECT column name 'embeddings'...")
video_count = add_videos_to_chromadb(df)
print(f"📊 Final result: {video_count} videos added")

🚀 Executing the function with CORRECT column name 'embeddings'...
🔄 Starting to add videos to ChromaDB...
✅ Row 0: Successfully processed - 'Scientists Can’t Explain What’...'
   Embedding length: 384
✅ Row 1: Successfully processed - '$138 Million Pirate Treasure F...'
   Embedding length: 384
✅ Row 2: Successfully processed - 'If You See a Rubber Band on Yo...'
   Embedding length: 384
❌ Error processing row 80: cannot convert float NaN to integer
❌ Error processing row 139: cannot convert float NaN to integer
❌ Error processing row 141: cannot convert float NaN to integer
❌ Error processing row 153: cannot convert float NaN to integer
❌ Error processing row 209: cannot convert float NaN to integer
❌ Error processing row 213: cannot convert float NaN to integer
❌ Error processing row 255: cannot convert float NaN to integer
❌ Error processing row 441: cannot convert float NaN to integer

📊 Summary:
✅ Processed 614 rows successfully
❌ Had 8 problematic rows
💾 Saving to ChromaDB...
🎉 S

In [24]:
# Check if data was successfully added
print("📊 Verifying ChromaDB data...")
total_count = collection.count()
print(f"✅ Total videos in ChromaDB: {total_count}")

# Peek at what was added
print("\n👀 Sample items from ChromaDB:")
sample_items = collection.peek(limit=3)

for i, (doc, metadata) in enumerate(zip(sample_items['documents'], sample_items['metadatas'])):
    print(f"\n🎯 Sample {i+1}:")
    print(f"   Title: {metadata['title']}")
    print(f"   Channel: {metadata['channel_title']}")
    print(f"   Views: {metadata['view_count']}")
    print(f"   Document preview: {doc[:100]}...")

📊 Verifying ChromaDB data...
✅ Total videos in ChromaDB: 614

👀 Sample items from ChromaDB:

🎯 Sample 1:
   Title: Scientists Can’t Explain What’s Happening on This Mountain in Tibet
   Channel: BRIGHT SIDE
   Views: 7095
   Document preview: Scientists Can’t Explain What’s Happening on This Mountain in Tibet Scientists Canâ€™t Explain Whatâ...

🎯 Sample 2:
   Title: $138 Million Pirate Treasure Found on the Ocean Floor
   Channel: BRIGHT SIDE
   Views: 4179
   Document preview: $138 Million Pirate Treasure Found on the Ocean Floor $138 Million Pirate Treasure Found on the Ocea...

🎯 Sample 3:
   Title: If You See a Rubber Band on Your Door, Act Fast!
   Channel: BRIGHT SIDE
   Views: 61970
   Document preview: If You See a Rubber Band on Your Door, Act Fast! If You See a Rubber Bads ...


In [27]:
import re

def convert_duration_to_seconds_manual(duration_str):
    """
    Convert ISO 8601 duration format to seconds without external library
    Examples: "PT5M30S" -> 330, "PT1H2M3S" -> 3723
    """
    try:
        # Extract hours, minutes, seconds using regex
        hours = re.search(r'(\d+)H', duration_str)
        minutes = re.search(r'(\d+)M', duration_str)
        seconds = re.search(r'(\d+)S', duration_str)
        
        total_seconds = 0
        if hours:
            total_seconds += int(hours.group(1)) * 3600
        if minutes:
            total_seconds += int(minutes.group(1)) * 60
        if seconds:
            total_seconds += int(seconds.group(1))
            
        return total_seconds
    except:
        return duration_str  # Return original if conversion fails

def search_similar_videos(query_text, n_results=5):
    """
    Search for similar videos using text query
    """
    print(f"🔍 Searching for: '{query_text}'")
    
    results = collection.query(
        query_texts=[query_text],
        n_results=n_results
    )
    
    print(f"🎯 Found {len(results['documents'][0])} results:")
    
    for i, (doc, metadata, distance) in enumerate(zip(
        results['documents'][0],
        results['metadatas'][0],
        results['distances'][0]
    )):
        # Convert duration to seconds
        duration_seconds = convert_duration_to_seconds_manual(metadata['duration'])
        
        # Format duration for display
        if isinstance(duration_seconds, int):
            duration_display = f"{duration_seconds} seconds"
            if duration_seconds >= 60:
                minutes = duration_seconds // 60
                remaining_seconds = duration_seconds % 60
                duration_display += f" ({minutes}m {remaining_seconds}s)"
        else:
            duration_display = metadata['duration']  # Original format
        
        print(f"\n📹 Result {i+1} (Similarity: {1-distance:.4f}):")
        print(f"   🎬 Title: {metadata['title']}")
        print(f"   📺 Channel: {metadata['channel_title']}")
        print(f"   👀 Views: {metadata['view_count']}")
        print(f"   ⏱️ Duration: {duration_display}")
        print(f"   📝 Preview: {doc[:100]}...")
    
    return results

# Test some searches
print("🚀 Testing Search Functionality with Duration in Seconds")
print("=" * 60)

# Test queries
test_queries = [
    "mountain Tibet mystery",
    "treasure shipwreck ocean", 
    "scientists discovery"
]

for query in test_queries:
    results = search_similar_videos(query, n_results=3)
    print("\n" + "=" * 60 + "\n")

🚀 Testing Search Functionality with Duration in Seconds
🔍 Searching for: 'mountain Tibet mystery'
🎯 Found 3 results:

📹 Result 1 (Similarity: 0.4027):
   🎬 Title: Scientists Can’t Explain What’s Happening on This Mountain in Tibet
   📺 Channel: BRIGHT SIDE
   👀 Views: 7095
   ⏱️ Duration: 14366 seconds (239m 26s)
   📝 Preview: Scientists Can’t Explain What’s Happening on This Mountain in Tibet Scientists Canâ€™t Explain Whatâ...

📹 Result 2 (Similarity: 0.0965):
   🎬 Title: New Mountain Taller Than 4 Burj Khalifas Just Found on Earth
   📺 Channel: BRIGHT SIDE
   👀 Views: 8088
   ⏱️ Duration: 536 seconds (8m 56s)
   📝 Preview: New Mountain Taller Than 4 Burj Khalifas Just Found on Earth New Mountain Taller Than 4 Burj Khalifa...

📹 Result 3 (Similarity: -0.3693):
   🎬 Title: A Giant Blob Is Slowly Heading for New York
   📺 Channel: BRIGHT SIDE
   👀 Views: 23146
   ⏱️ Duration: 10889 seconds (181m 29s)
   📝 Preview: A Giant Blob Is Slowly Heading for New York A Giant Blob Is Slowly Headi

In [30]:
def search_with_filters(query_text, min_views=None, channel=None, n_results=5):
    """
    Advanced search with filters
    """
    where_filter = {}
    
    if min_views is not None:
        where_filter["view_count"] = {"$gte": min_views}
    
    if channel is not None:
        where_filter["channel_title"] = {"$eq": channel}
    
    print(f"🔍 Advanced search: '{query_text}'")
    if where_filter:
        print(f"   Filters: {where_filter}")
    
    results = collection.query(
        query_texts=[query_text],
        n_results=n_results,
        where=where_filter
    )
    
    print(f"🎯 Found {len(results['documents'][0])} results:")
    for i, (doc, metadata, distance) in enumerate(zip(
        results['documents'][0],
        results['metadatas'][0],
        results['distances'][0]
    )):
        print(f"\n📹 Result {i+1} (Similarity: {1-distance:.4f}):")
        print(f"   🎬 Title: {metadata['title']}")
        print(f"   📺 Channel: {metadata['channel_title']}")
        print(f"   👀 Views: {metadata['view_count']}")
    
    return results

# Test filtered searches
print("🎛️ Testing Filtered Searches")
print("=" * 50)

# Search for high-view videos about mysteries
search_with_filters("mystery", min_views=10000, n_results=3)
print("\n" + "=" * 50)

# You can also filter by specific channel if you know channel names
# search_with_filters("treasure", channel="Bright Side", n_results=3)

🎛️ Testing Filtered Searches
🔍 Advanced search: 'mystery'
   Filters: {'view_count': {'$gte': 10000}}
🎯 Found 3 results:

📹 Result 1 (Similarity: -0.2221):
   🎬 Title: Scientists Just Solved the Mystery of Killer Waves
   📺 Channel: BRIGHT SIDE
   👀 Views: 10632

📹 Result 2 (Similarity: -0.2743):
   🎬 Title: Weirdest Things People Actually Found in the Desert
   📺 Channel: BRIGHT SIDE
   👀 Views: 18526

📹 Result 3 (Similarity: -0.4140):
   🎬 Title: Dark Food Joke #Comedy #Shorts
   📺 Channel: Saikiran
   👀 Views: 15429



In [32]:
def debug_metadata_structure():
    """
    Debug what metadata structure actually exists
    """
    print("🔍 Debugging Metadata Structure")
    print("=" * 40)
    
    total_count = collection.count()
    print(f"Total items: {total_count}")
    
    # Get a few sample items
    sample_items = collection.peek(limit=3)
    
    for i, metadata in enumerate(sample_items['metadatas']):
        print(f"\n📄 Sample {i+1} Metadata:")
        print(f"   Type: {type(metadata)}")
        print(f"   Keys: {list(metadata.keys())}")
        for key, value in metadata.items():
            print(f"   - {key}: {value}")

# Run debug first
debug_metadata_structure()

🔍 Debugging Metadata Structure
Total items: 614

📄 Sample 1 Metadata:
   Type: <class 'dict'>
   Keys: ['video_id', 'like_count', 'view_count', 'published_at', 'title', 'description', 'channel_title', 'duration']
   - video_id: t8txtQkhMcY
   - like_count: 202
   - view_count: 7095
   - published_at: 2025-10-28T06:45:00Z
   - title: Scientists Can’t Explain What’s Happening on This Mountain in Tibet
   - description: High in Tibet stands a mountain no one has ever conquered — Mount Kailash. For centuries, climbers have failed to reach its peak, and even scientists can’t fully explain why. Locals say it’s sacred, o
   - channel_title: BRIGHT SIDE
   - duration: PT3H59M26S

📄 Sample 2 Metadata:
   Type: <class 'dict'>
   Keys: ['channel_title', 'title', 'duration', 'video_id', 'view_count', 'like_count', 'published_at', 'description']
   - channel_title: BRIGHT SIDE
   - title: $138 Million Pirate Treasure Found on the Ocean Floor
   - duration: PT8M50S
   - video_id: mmIidOoHCfs
   - vi

In [33]:
def get_collection_stats():
    """
    Get detailed statistics about the collection with error handling
    """
    print("📈 ChromaDB Collection Statistics")
    print("=" * 40)
    
    total_count = collection.count()
    print(f"📊 Total videos: {total_count}")
    
    # Get all items to calculate stats
    all_items = collection.get(limit=total_count)
    
    # View count statistics with error handling
    view_counts = []
    for item in all_items['metadatas']:
        try:
            # Use get() method to handle missing keys safely
            view_count = item.get('view_count', 0)
            if isinstance(view_count, (int, float)):
                view_counts.append(view_count)
        except:
            continue
    
    if view_counts:
        print(f"👀 Total views across all videos: {sum(view_counts):,}")
        print(f"📈 Average views per video: {sum(view_counts) // len(view_counts):,}")
        print(f"🔥 Most viewed video: {max(view_counts):,} views")
        print(f"😴 Least viewed video: {min(view_counts):,} views")
    else:
        print("👀 No view count data available")
    
    # Channel statistics with error handling
    channels = []
    for item in all_items['metadatas']:
        try:
            # Safely get channel_title, provide default if missing
            channel = item.get('channel_title', 'Unknown Channel')
            if channel and channel != 'Unknown Channel':
                channels.append(channel)
        except:
            continue
    
    if channels:
        unique_channels = set(channels)
        print(f"📺 Unique channels: {len(unique_channels)}")
        # Show top 5 channels by occurrence
        from collections import Counter
        channel_counts = Counter(channels)
        top_channels = channel_counts.most_common(5)
        print(f"🏆 Top 5 channels:")
        for channel, count in top_channels:
            print(f"   - {channel}: {count} videos")
    else:
        print("📺 No channel data available")
    
    # Duration statistics
    durations = []
    for item in all_items['metadatas']:
        try:
            duration = item.get('duration', '')
            if duration:
                durations.append(duration)
        except:
            continue
    
    print(f"⏱️ Videos with duration data: {len(durations)}")
    
    # Check metadata keys to understand what's available
    if all_items['metadatas']:
        sample_metadata = all_items['metadatas'][0]
        print(f"🔑 Available metadata keys: {list(sample_metadata.keys())}")

# Run the robust version
get_collection_stats()

📈 ChromaDB Collection Statistics
📊 Total videos: 614
👀 Total views across all videos: 83,965,023
📈 Average views per video: 136,750
🔥 Most viewed video: 3,835,647 views
😴 Least viewed video: 40 views
📺 Unique channels: 16
🏆 Top 5 channels:
   - Telusko: 50 videos
   - ProgrammingWithHarry: 49 videos
   - Saikiran: 47 videos
   - Fireship: 47 videos
   - BRIGHT SIDE: 44 videos
⏱️ Videos with duration data: 614
🔑 Available metadata keys: ['duration', 'description', 'view_count', 'title', 'like_count', 'channel_title', 'published_at', 'video_id']


In [34]:
# This is what you can show your mentor!
def demo_for_mentor():
    """
    Complete demonstration of the vector search capabilities
    """
    print("🎉 CHROMADB VECTOR SEARCH DEMONSTRATION")
    print("=" * 60)
    
    # 1. Show we have data
    print("1. 📊 DATA LOADED SUCCESSFULLY")
    print(f"   - Total videos: {collection.count()}")
    print(f"   - Each video has: Title, Transcript, Views, Channel, Embeddings")
    
    # 2. Demonstrate semantic search
    print("\n2. 🔍 SEMANTIC SEARCH DEMO")
    print("   Searching for 'ancient mysteries'...")
    results = search_similar_videos("ancient mysteries", n_results=2)
    
    # 3. Demonstrate filtered search
    print("\n3. 🎛️ FILTERED SEARCH DEMO")
    print("   Searching for 'treasure' with > 10K views...")
    search_with_filters("treasure", min_views=10000, n_results=2)
    
    # 4. Explain the technology stack
    print("\n4. 🛠️ TECHNOLOGY STACK")
    print("   - Embedding Model: all-MiniLM-L6-v2 (384-dimensional vectors)")
    print("   - Vector Database: ChromaDB (optimized for similarity search)")
    print("   - Search Type: Semantic search using cosine similarity")
    print("   - Metadata: Full filtering capability (views, channel, etc.)")
    
    print("\n✅ DEMONSTRATION COMPLETE!")
    print("🎯 The system can now find semantically similar videos instantly!")

# Run the demonstration
demo_for_mentor()

🎉 CHROMADB VECTOR SEARCH DEMONSTRATION
1. 📊 DATA LOADED SUCCESSFULLY
   - Total videos: 614
   - Each video has: Title, Transcript, Views, Channel, Embeddings

2. 🔍 SEMANTIC SEARCH DEMO
   Searching for 'ancient mysteries'...
🔍 Searching for: 'ancient mysteries'
🎯 Found 2 results:

📹 Result 1 (Similarity: 0.0745):
   🎬 Title: Cleopatra’s Tomb Found? Egypt’s Greatest Discovery in 2,000 Years
   📺 Channel: BRIGHT SIDE
   👀 Views: 4010
   ⏱️ Duration: 11065 seconds (184m 25s)
   📝 Preview: Cleopatra’s Tomb Found? Egypt’s Greatest Discovery in 2,000 Years Cleopatraâ€™s Tomb Found? Egyptâ€™...

📹 Result 2 (Similarity: -0.1156):
   🎬 Title: Weirdest Things People Actually Found in the Desert
   📺 Channel: BRIGHT SIDE
   👀 Views: 18526
   ⏱️ Duration: 10827 seconds (180m 27s)
   📝 Preview: Weirdest Things People Actually Found in the Desert Weirdest Things People Actually Found in the Des...

3. 🎛️ FILTERED SEARCH DEMO
   Searching for 'treasure' with > 10K views...
🔍 Advanced search: 'treasu